In [ ]:
#Цель проекта:
#Выбрать параметры датасета для модели, которая предсказывала бы результаты госэкзамена по математике.

In [ ]:
#Задачи проекта:
#Провести первичную обработку данных;
#По необходимости преобразовать данные;
#Провести разведывательный анализ данных;
#Выявить параметры, сильно влияющие на успеваемость по математике;
#Создать датасет с выбранными параметры для модели;
#Составить отчёт по результатам работы.

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.simplefilter('ignore')
from itertools import combinations
from scipy.stats import ttest_ind
pd.set_option('display.max_columns', 50) 

In [ ]:
#Функции
#Оценка количества уникальных значений для каждого столбца в датасете:
def unic_data(dataset):
    for column in dataset:
        display(column)
        display(dataset.loc[:, column].unique())
        display(pd.DataFrame(dataset.loc[:, column].value_counts()))
#Создание распределение оценки для каждого параметра с помощью Boxplot:  
def get_boxplot(column):
    if column == 'score':
        return 0 
    else:
        fig, ax = plt.subplots(figsize = (14, 4))
        sns.boxplot(x=column, y='score', 
                    data=students.loc[students.loc[:, column].isin(students.loc[:, column].value_counts().index[:20])],
                   ax=ax)
        plt.xticks(rotation=45)
        ax.set_title('Boxplot for ' + column)
        plt.show()
#Распределение оценок по номинативным признакам, с помощью теста Стьюдента:
def get_stat_dif(column):
    marker = 0
    if column == 'score':
        return 0 
    else:
        cols = students.loc[:, column].value_counts().index[:20]
        combinations_all = list(combinations(cols, 2))
        for comb in combinations_all:
            if ttest_ind(students.loc[students.loc[:, column] == comb[0], 'score'],
                         students.loc[students.loc[:, column] == comb[1], 'score'],
                         nan_policy='omit').pvalue <= 0.1/len(combinations_all):
                print('Найдены статистически значимые различия для колонки', column)
                marker = 1
                break
    return marker

In [ ]:
#Прочитаем файл и осмотрим данные:
students = pd.read_csv('stud_math.csv')
students

In [ ]:
#Выведем информацию о датасете:
students.info()

In [ ]:
#В описании данных не сказано о столбце 'studytime, granular'.
#Возможно стоит его удалить, т.к описать влияние неизветного параметра - невыполнимая задача.
#Но оставим его неизменным и посмотрим корреляцию - ведь стереть неизвестный параметр всегда успеем.

In [ ]:
#Сохраним копию изначального датасета перед преобразованием:
data = students.copy()

In [ ]:
#Оценим количество уникальных значений для каждого столбца:
unic_data(students)

In [ ]:
#Расмотрим пропуски в столбцах и возможность их восстановить:
#Пропуски не восстановить для следующих параметров:
#Адрес (address);
#Состав семьи (famsize);
#Состоянии семьи (Pstatus);
#Образование и работа родителей (Medu, Fedu, Mjob, Fjob);
#Опекунство (guardian);
#Время в пути до школы (traveltime);
#Время на учёбу помимо школы (studytime);
#Внеучебные неудачи (failures);
#Посещение детского сада (nursery);
#Желание о высшем обравзовании (higher);
#Семейные отношения (famrel);
#Свободное время (freetime);
#Время с друзьями (goout);
#Здоровье (health);
#Количество пропущенных занятий (absences);
#Баллы по математике (score).

In [ ]:
#Пропуски причины выбора школы (reason) можнозаполнить причиной "other":
students.reason = students.reason.astype(str).apply(lambda x: "other" if x.strip() == '' else "other" if x == 'nan'  else x)

In [ ]:
#Пропуски данных, для которых можем допустить, что их нет и заменить на соответствуещее значение ('no'):
#Дополнительной школьной (schoolsup) и семейной (famsup) образовательных поддержек;
#Дополнительные занятия (paid);
#Внеучебные занятия (activities);
#Наличие интернета (internet) 
#Наличие романтических отношений (romantic) 
for col in ['schoolsup', 
            'famsup', 
            'paid', 
            'activities',
            'internet', 
            'romantic']:
    students.loc[:, col] = students.loc[:, col].astype(str).apply(lambda x: 'no' if x.strip() == '' else 'no' if x == 'nan' else x)

In [ ]:
#Посмотрим на распределение признака для числовых переменных с помощью boxplot:
for col in ['age', 
            'Medu', 
            'Fedu', 
            'traveltime', 
            'studytime', 
            'failures', 
            'studytime, granular', 
            'famrel', 
            'freetime', 
            'goout', 
            'health', 
            'absences', 
            'score']:
        students.loc[:, col].plot(kind='box', grid=True)
        plt.show()

In [ ]:
#Рассмотрим выбросы:
#Возраст (age) 22 года - это не выброс. Оставим в датасете.
#Образование отца (Fedu) равно 40 - это явный выброс. Заменим на 'None':
students.loc[students['Fedu'] > 4,'Fedu'] = None
#График boxplot количества внеучебных неудач (failures) объясняется модой значений равной 0.
#Значения внеучебных неудач (failures) отличные от 0 - это не выбросы.
#Cемейные отношения (famrel) равное -1 - это явный выброс. Заменим на 'None':
students.loc[students['famrel'] < 0,'famrel'] = None
#График количества пропущенных занятий (absences) содержит явные выбросы с большим количеством прогулов. 
#С этими людьми нужно работать отдельно:
truants = students[students.absences > 50]
#В результате работы выделим отдельно этот датасет, убирая прогульщиков из основного датасета:
students = students[students.absences <= 50]
#По распределению признака для числовых переменных заметно большое количество оценок 0 для студентов (37 значений). 
#Создадим для них отдельный датасет:
bad_result = students[students.score == 0]
#Возможно и стоит их рассмотреть отдельно, в результате работы выделим этот момент. 
#Но из основного датасета их убирать не будем.
#Также удалим данные с неизвестной оценкой:
students = students[students.score != None]

In [ ]:
#Проведем корреляционный анализ количественных переменных:
students.corr()

In [ ]:
#Самые не коррелирующие с оценкой данные: 
#Семейные отношения (famrel); 
#Свободное время (freetime);
#Здоровье (health);
#Прогулы (absences).

In [ ]:
#Самый заметный фактор влияющий на оценку: 
#Обратная кореляция с количеством жизненных неудач - чем неудач больше, тем ниже оценка.

In [ ]:
#Посмотрим на распределение оценки (score) для каждого параметра с помощью boxplot:
for col in students:
    get_boxplot(col)

In [ ]:
#Среди факторов не влияющих на оценку можно выделить следующие параметры:
#Работа отца (Fjob);
#Причина выбора школы (reason); 
#Внеучебные занятия (activities); 
#Посещение детского сада (nursery).

In [ ]:
#На графиках явно видно, что прогулы (absences) влияют на оценку, несмотря на корреляционный анализ
#Большое количество прогулов сильно снижает среднюю оценку.

In [ ]:
#Создадим датасет модели и запишем в него оценку (score):
students_model = pd.DataFrame({})
students_model['score'] = students['score']

In [ ]:
#Проанализируем переменные которые влияют на итоговую оценку (score):
for col in students:
    check = get_stat_dif(col)
    if check == 1:
        students_model.loc[:, [col]] = students.loc[:, [col]]
        #В модель запишем переменные которые влияют на оценку (score):

In [ ]:
#Проверим, отличаются ли данные для первоначального датасета и преобразованного:
data_check = data[data.score != None]
for col in data_check:
    check = get_stat_dif(col)
#Так мы убедимся, что не потеряли ни один важный параметр из-за преобразования данных.

In [ ]:
#Как видим выбранные параметры для преобразованного датасета не отличаются от первоначального.

In [ ]:
#Осмотрим полученные данные для модели:
students_model

In [ ]:
# Результаты работы
#Преобразованы данные, пропущеные данные заменены, сохранен не преобразованный датасет (data);
#Устранены выбросы, также из датасета (students) убраны прогульщики (больше 50 прогулов) и сохранены в 
#отдельный датасет (truants);
#Стоит обратит внимание на большое количество нулевых результатов. Из датасета (students) они не были 
#убраны, но создан отдельный датасет (bad_result);
#Выбраны параметры, сильно влияющие на успеваемость по математике;
#Выбранные параметры для преобразованного датасета (students) не отличаются от первоначального (data).
#Создан датасет с выбранными параметрами для модели (students_model).